In [ ]:
#!/home/xyang18/miniconda3/envs/pytorch/bin/ python
# -*- coding: utf-8 -*-
# Python version: 3.6

import os
import numpy as np
import pickle as cp
import torch
from torch import nn
from torch.nn import functional as F
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, BatchNorm1d, Dropout, Flatten, BCELoss
from torch.utils.data import TensorDataset

from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import sklearn.metrics as metrics
from sliding_window import sliding_window

In [ ]:
gpu_id=0
if gpu_id>=0:
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
    cuda_id = "cuda:" + str(0)  # cuda:2

device = torch.device(cuda_id if torch.cuda.is_available() else "cpu")
print("Device:", device)
if (torch.cuda.is_available()):
    torch.cuda.set_device(cuda_id)
    print("Current GPU ID:", torch.cuda.current_device())

## check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

In [ ]:
# Number of Sensor Channels used in the OPPORTUNITY dataset.
NB_SENSOR_CHANNELS = 113

# Number of classes in which data is classified (or to be classified).
NUM_CLASSES = 5

# Length of the sliding window used to segmenting the time-series-data.
SLIDING_WINDOW_LENGTH = 24

# Steps of the sliding window used in segmenting the data.
SLIDING_WINDOW_STEP = 12

act_labels_txt = ['std', 'wlk', 'sit', 'lie', 'null']

In [ ]:
def load_dataset(filename):

    f = open(filename, 'rb')
    data = cp.load(f)
    f.close()

    X_train, y_train = data[0]
    X_test, y_test = data[1]

    print(" ..from file {}".format(filename))
    print(" ..reading instances: train {0}, test {1}".format(X_train.shape, X_test.shape))

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # The targets are casted to int8 for GPU compatibility.
    y_train = y_train.astype(np.uint8)
    y_test = y_test.astype(np.uint8)

    return X_train, y_train, X_test, y_test

print("Loading Data...")
X_train, y_train, X_test, y_test = load_dataset('../../data/oppChallenge_gestures.data')

assert NB_SENSOR_CHANNELS == X_train.shape[1]
def opp_sliding_window(data_x, data_y, ws, ss):
    data_x = sliding_window(data_x,(ws,data_x.shape[1]),(ss,1))
    data_y = np.asarray([[i[-1]] for i in sliding_window(data_y,ws,ss)])
    return data_x.astype(np.float32), data_y.reshape(len(data_y)).astype(np.uint8)

# Sensor data is segmented using a sliding window mechanism
X_test, y_test = opp_sliding_window(X_test, y_test, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
print(" ..after sliding window (testing): inputs {0}, targets {1}".format(X_test.shape, y_test.shape))

# Data is reshaped since the input of the network is a 4 dimension tensor
X_test = X_test.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS))

In [ ]:
X_train, y_train = opp_sliding_window(X_train, y_train, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
print(" ..after sliding window (training): inputs {0}, targets {1}".format(X_train.shape, y_train.shape))
X_train = X_train.reshape((-1,SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS))

In [ ]:
y_train = to_categorical(y_train, num_classes=NUM_CLASSES)
y_test = to_categorical(y_test, num_classes=NUM_CLASSES)

In [ ]:
class HARModel(nn.Module):
    
    def __init__(self, n_hidden=128, n_layers=1, n_filters=64, 
                 n_classes=5, filter_size=(1,5), drop_prob=0.5):
        super(HARModel, self).__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.n_filters = n_filters
        self.n_classes = n_classes
        self.filter_size = filter_size
             
        self.conv1 = nn.Conv2d(1, n_filters, filter_size)
        self.conv2 = nn.Conv2d(n_filters, n_filters, filter_size)
        self.conv3 = nn.Conv2d(n_filters, n_filters, filter_size)
        self.conv4 = nn.Conv2d(n_filters, n_filters, filter_size)
        self.lstm1  = nn.LSTM(7232, n_hidden, n_layers)
        self.lstm2  = nn.LSTM(n_hidden, n_hidden, n_layers)
        self.fc = nn.Linear(1024, n_classes)
        self.dropout = nn.Dropout(drop_prob)
    
    def forward(self, x, hidden, batch_size):
        x = torch.permute(x, (0,2,1))
        x = torch.unsqueeze(x, dim=1)        
        
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))

        x = torch.permute(x, (3,0,1,2))
        x = x.view(x.shape[0], x.shape[1],-1)      
        
        x, hidden = self.lstm1(x, hidden)
        x = self.dropout(x)
        x, hidden = self.lstm2(x, hidden)
        x = self.dropout(x)

        x = torch.permute(x, (1,0,2))
        x = torch.reshape(x, (batch_size,-1))

        out = self.fc(x)        
        return out, hidden
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
    
net = HARModel()

In [ ]:
def init_weights(m):
    if type(m) == nn.LSTM:
        for name, param in m.named_parameters():
            if 'weight_ih' in name:
                torch.nn.init.orthogonal_(param.data)
            elif 'weight_hh' in name:
                torch.nn.init.orthogonal_(param.data)
            elif 'bias' in name:
                param.data.fill_(0)
    elif type(m) == nn.Conv1d or type(m) == nn.Linear:
        torch.nn.init.orthogonal_(m.weight)
        m.bias.data.fill_(0)
net.apply(init_weights)    

In [ ]:
def train(net, epochs=20, batch_size=64, lr=0.01):
    opt = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=1e-4)

    criterion = nn.CrossEntropyLoss()

    train_dataset = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
    train_loader = torch.utils.data.DataLoader(train_dataset,
        batch_size=batch_size, shuffle=True, drop_last = True)  

    test_dataset = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
    test_loader = torch.utils.data.DataLoader(test_dataset,
        batch_size=batch_size, shuffle=False, drop_last = True) 
    
    
    if(train_on_gpu):
        net.cuda()
     
    for e in range(epochs):
        
        # initialize hidden state
        h = net.init_hidden(batch_size)         
        train_losses = []    
        net.train()

        for batch in train_loader:
            x, y = batch
            inputs, targets = x.to(device), y.to(device)  
            h = tuple([each.data for each in h])
            opt.zero_grad()   
            output, h = net(inputs, h, batch_size)

            loss = criterion(output, torch.argmax(targets,dim=1))
            train_losses.append(loss.item())
            loss.backward()
            opt.step()
            
        val_h = net.init_hidden(batch_size)
        val_losses = []
        accuracy=0
        f1score=0
        
        correct = 0
        total = 0
        total_true = []
        total_pred = []
        
        net.eval()

        with torch.no_grad():
            for batch in test_loader:
                x, y = batch
                inputs, targets = x.to(device), y.to(device)  
                
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, targets = inputs.cuda(), targets.cuda()
                    
                output, val_h= net(inputs, val_h, batch_size)

                val_loss = criterion(output, torch.argmax(targets,dim=1))
                val_losses.append(val_loss.item())

                predicted = torch.argmax(output.data, dim=1)
                total += targets.size(0)
                correct += (predicted == torch.argmax(targets, dim=1)).sum().item()

                total_pred = total_pred + predicted.cpu().numpy().tolist()
                total_true = total_true + (torch.argmax(targets, dim=1).cpu().numpy().tolist())                
                
        net.train() # reset to train mode after iterationg through validation data
                
        f1_score = metrics.f1_score(y_true = total_true, y_pred = total_pred, average='weighted')     

        print("Epoch: {}/{}...".format(e+1, epochs),
        "Train Loss: {:.4f}...".format(np.mean(train_losses)),
        "Val Loss: {:.4f}...".format(np.mean(val_losses)),
        "Val Acc: {:.4f}...".format(correct / total),
        "F1-Score: {:.4f}...".format(f1_score))
        
train(net)

In [ ]:
PATH = 'opportunity_deepConvLSTM.pt'
torch.save(net.state_dict(), PATH)